Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [13]:
from random import random, seed
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from icecream import ic


## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [14]:
# UNIVERSE_SIZE = 100
# NUM_SETS = 10
# DENSITY = 0.2  # how dense are the sets (low density means each set has few elements)

# rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [15]:
# DON'T EDIT THESE LINES!

# SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
# for s in range(UNIVERSE_SIZE):
#     if not np.any(SETS[:, s]):
#         SETS[np.random.randint(NUM_SETS), s] = True
# COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [16]:
# def valid(solution):
def valid(solution, sets):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(sets[solution]))


def cost(solution, costs):
    """Returns the cost of a solution (to be minimized)"""
    return costs[solution].sum()

## Have Fun!

In [17]:
# A dumb solution of "all" sets
# solution = np.full(NUM_SETS, True)
# valid(solution), cost(solution)

In [18]:
# A random solution with random 50% of the sets
# solution = rng.random(NUM_SETS) < .5
# valid(solution), cost(solution)

In [19]:
def fitness(solution, sets, costs): # fitness is something to be maximized, minimizing cost instead
    return (valid(solution,sets),-cost(solution, costs) )


## Solution - 1 - Random Hill Climbing

In [20]:
def hill_climber(solution, SETS, COSTS, NUM_SETS, max_steps):
    # Inizializzazione: Soluzione vuota, ovvero nessun sottoinsieme selezionato
    #best_value = float('inf')  # Cost iniziale alto (perché vogliamo minimizzare)
    history = [valid(solution, SETS)]

    for step in tqdm(range(max_steps)):
        # Crea una nuova soluzione facendo il tweak 
        new_solution = solution.copy()
        index = np.random.randint(0, NUM_SETS)
        new_solution[index] = not new_solution[index]  # Aggiungi o rimuovi il sottoinsieme selezionato
        history.append(valid(new_solution, SETS))
        if fitness(new_solution, SETS, COSTS) > fitness(solution, SETS, COSTS):
            solution = new_solution

    ic(fitness(solution, SETS, COSTS))
    return solution, history


In [21]:
def simulated_annealing(solution, SETS, COSTS, NUM_SETS, max_steps, initial_temp, cooling_rate):
    current_solution = solution.copy()
    current_fitness = fitness(current_solution, SETS, COSTS)
    best_solution = current_solution
    best_fitness = current_fitness
    temp = initial_temp
    
    history = [current_fitness]

    for step in tqdm(range(max_steps)):
        if temp <= 0:
            break
        new_solution = current_solution.copy()
        index = np.random.randint(0, NUM_SETS)
        new_solution[index] = not new_solution[index]
        new_fitness = fitness(new_solution, SETS, COSTS)
        
        # Se la nuova soluzione è migliore, la accettiamo
        if new_fitness > current_fitness:
            current_solution = new_solution
            current_fitness = new_fitness
        else:
            # Altrimenti accettiamo con una probabilità basata sulla temperatura
            delta = new_fitness[1] - current_fitness[1]  # confronto sui costi
            if np.random.rand() < np.exp(delta / temp):
                current_solution = new_solution
                current_fitness = new_fitness
        
        # Raffreddamento della temperatura
        temp *= cooling_rate
        
        history.append(current_fitness)
        if current_fitness > best_fitness:
            best_solution = current_solution
            best_fitness = current_fitness
    
    return best_solution, history


In [22]:
def tweak(solution, NUM_SETS):
    """Effettua una piccola modifica alla soluzione corrente"""
    new_solution = solution.copy()
    index = np.random.randint(0, NUM_SETS)
    new_solution[index] = not new_solution[index]
    return new_solution


In [23]:
def steepest_ascent_with_restart(SETS, COSTS, NUM_SETS, max_steps, num_restarts, candidates_per_step):
    best_solution = None
    max_value = -float('inf')  # Partiamo dal peggiore valore possibile
    num_steps = 0
    history = list()

    for r in tqdm(range(num_restarts), desc="Restarts"):
        # Inizializza la soluzione per questo restart
        solution = np.random.choice([True, False], size=NUM_SETS)
        history.append(valid(solution, SETS))

        for step in tqdm(range(max_steps), desc="Steps"):
            # Genera una serie di soluzioni candidate (tweak della soluzione attuale)
            candidates = [tweak(solution, NUM_SETS) for _ in range(candidates_per_step)]
            candidates_fitness = [fitness(c, SETS, COSTS) for c in candidates]

            # Trova la soluzione con il miglioramento più ripido
            idx = np.argmax([f[0] for f in candidates_fitness])
            new_solution = candidates[idx]
            new_fitness = candidates_fitness[idx]

            num_steps += candidates_per_step
            history.append(new_fitness[0])

            # Se la nuova soluzione è migliore, aggiorna la soluzione corrente
            if new_fitness > fitness(solution, SETS, COSTS):
                solution = new_solution

        # Se la soluzione di questo restart è la migliore trovata, aggiorna la soluzione globale
        if fitness(solution, SETS, COSTS)[1] > max_value:  # [1] è il costo negativo
            max_value = fitness(solution, SETS, COSTS)[1]  # Aggiorniamo solo il valore del costo
            best_solution = solution

    return best_solution, history


In [24]:
# Funzione per generare i set di dati
def generate_sets(universe_size, num_sets, density):
    # Generazione dei sottoinsiemi e costi
    SETS = np.random.random((num_sets, universe_size)) < density
    for s in range(universe_size):
        if not np.any(SETS[:, s]):
            SETS[np.random.randint(num_sets), s] = True
    COSTS = np.pow(SETS.sum(axis=1), 1.1)
    return SETS, COSTS


# Funzione per eseguire i test per ciascuna combinazione di parametri
def run_experiments():

    instances = [
        {"Universe size": 100, "Num sets": 10, "Density": 0.2},
        {"Universe size": 1_000, "Num sets": 100, "Density": 0.2},
        {"Universe size": 10_000, "Num sets": 1_000, "Density": 0.2},
        {"Universe size": 100_000, "Num sets": 10_000, "Density": 0.1},
        {"Universe size": 100_000, "Num sets": 10_000, "Density": 0.2},
        {"Universe size": 100_000, "Num sets": 10_000, "Density": 0.3}
    ]
    
    max_steps = 1000
    candidates_per_step = 3
    num_restarts = 5

     # Parametri per Simulated Annealing
    initial_temp = 1000
    cooling_rate = 0.995

    # Eseguire gli algoritmi per ogni combinazione
    for instance in instances:
        universe_size = instance["Universe size"]
        num_sets = instance["Num sets"]
        density = instance["Density"]
        print(f"\n--- Running for Universe size={universe_size}, Num sets={num_sets}, Density={density} ---")
        rng = np.random.Generator(np.random.PCG64([universe_size, num_sets, int(10_000 * density)]))
        # Genera il dataset
        SETS, COSTS = generate_sets(universe_size, num_sets, density)
        solution = rng.random(num_sets) < 1
        # Esecuzione degli algoritmi
        print("Running Hill Climber...")
        hc_solution, hc_history = hill_climber(solution, SETS, COSTS, num_sets, max_steps)
        print("Running Simulated Annealing...")
        sa_solution, sa_history = simulated_annealing(solution, SETS, COSTS, num_sets, max_steps, initial_temp, cooling_rate)
        print("Running Steepest Ascent with Restarts...")
        sar_solution, sar_history = steepest_ascent_with_restart(SETS, COSTS, num_sets, max_steps, num_restarts, candidates_per_step)
        
        # Stampa delle migliori soluzioni
        print(f"Hill Climber best solution cost: {cost(hc_solution, COSTS)}")
        print(f"Simulated Annealing best solution cost: {cost(sa_solution, COSTS)}")
        print(f"Steepest Ascent with Restarts best solution cost: {cost(sar_solution, COSTS)}")


        # # Plot dei risultati
        # plt.figure(figsize=(12, 6))
        # plt.plot(hc_history, label="Hill Climber", color="blue")
        # plt.plot(sa_history, label="Steepest Ascent", color="green")
        # plt.plot(hcr_history, label="Hill Climber with Restarts", color="red")
        # plt.xlabel("Steps")
        # plt.ylabel("Cost")
        # plt.legend()
        # plt.title(f"Cost Reduction for Universe size={universe_size}, Num sets={num_sets}, Density={density}")
        # plt.grid(True)
        # plt.show()

# Esegui il confronto tra gli algoritmi su tutte le combinazioni
run_experiments()


--- Running for Universe size=100, Num sets=10, Density=0.2 ---
Running Hill Climber...


  0%|          | 0/1000 [00:00<?, ?it/s]

ic| fitness(solution, SETS, COSTS): (np.True_, np.float64(-304.4003633618365))


Running Simulated Annealing...


  0%|          | 0/1000 [00:00<?, ?it/s]

...

Steps:   0%|          | 0/1000 [00:00<?, ?it/s]

Steps:   0%|          | 0/1000 [00:00<?, ?it/s]

Steps:   0%|          | 0/1000 [00:00<?, ?it/s]

Steps:   0%|          | 0/1000 [00:00<?, ?it/s]

Hill Climber best solution cost: 761222597.2389413
Simulated Annealing best solution cost: 761712495.0806645
Steepest Ascent with Restarts best solution cost: 375370696.3544909
